In [1]:
import findspark
findspark.init()
from pyspark.context import SparkContext, SparkConf
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType, ArrayType, StringType, ShortType, FloatType
from pyspark.sql import SparkSession
from spark_functions import *

c:\Users\39392\anaconda3\envs\spark_env\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
sc = SparkContext().getOrCreate()

In [3]:
spark = SparkSession(sc).builder \
    .appName("medicitalia_doctors") \
    .getOrCreate()
    #.master("local[1]") \
print(spark)

In [4]:
category_mapping = {
    "Scienza dell'alimentazione": "Scienza dell'alimentazione",
    "Igiene e medicina preventiva": "Igiene e medicina preventiva",
    "Chirurgia maxillo-facciale": "Chirurgia maxillo-facciale",
    "Medicina fisica e riabilitativa": "Medicina fisica e riabilitativa",
    "Oculistica": "Oftalmologia",
    "Cardiologia": "Cardiologia",
    "Andrologia": "Andrologia",
    "Neuroradiologia": "Neuroradiologia",
    "Chirurgia dell'apparato digerente": "Chirurgia dell'apparato digerente",
    "Chirurgia pediatrica": "Chirurgia pediatrica",
    "Microbiologia e virologia": "Microbiologia e virologia",
    "Geriatria": "Geriatria",
    "Diabetologia e malattie del metabolismo": "Diabetologia e malattie del metabolismo",
    "Medicina legale e delle assicurazioni": "Medicina legale e delle assicurazioni",
    "Tossicologia Medica": "Tossicologia Medica",
    "Oncologia medica": "Oncologia medica",
    "Odontoiatria e odontostomatologia": "Odontoiatria e odontostomatologia",
    "Endocrinologia": "Endocrinologia",
    "Ortodonzia": "Ortodonzia",
    "Urologia": "Urologia",
    "Ortopedia": "Ortopedia",
    "Otorinolaringoiatria": "Otorinolaringoiatria",
    "Chirurgia della mano": "Chirurgia della mano",
    "Medicina generale": "Medicina generale",
    "Pediatria": "Pediatria",
    "Medicine non convenzionali": "Medicine non convenzionali",
    "Senologia": "Senologia",
    "Cure palliative": "Cure palliative",
    "Neurofisiopatologia": "Neurofisiopatologia",
    "Psicologia Clinica": "Psicologia Clinica",
    "Chirurgia di urgenza e Pronto Soccorso": "Chirurgia di urgenza e Pronto Soccorso",
    "Ematologia": "Ematologia",
    "Neurochirurgia": "Neurochirurgia",
    "Audiologia e foniatria": "Audiologia e foniatria",
    "Radiologia": "Radiologia",
    "Nefrologia": "Nefrologia",
    "Medicina di base": "Medicina di base",
    "Psicoterapia": "Psicoterapia",
    "Pneumologia": "Pneumologia",
    "Medicina dello sport": "Medicina dello sport",
    "Psicologia": "Psicologia",
    "Biochimica clinica": "Biochimica clinica",
    "Patologia della riproduzione umana": "Patologia della riproduzione umana",
    "Chirurgia vascolare e angiologia": "Chirurgia vascolare e angiologia",
    "Allergologia e immunologia": "Allergologia e immunologia",
    "Medicina di laboratorio": "Medicina di laboratorio",
    "Angiologia": "Chirurgia vascolare e angiologia",
    "Medicina estetica": "Medicina estetica",
    "Cardiologia interventistica": "Cardiologia interventistica",
    "Genetica medica": "Genetica medica",
    "Endocrinochirurgia": "Endocrinochirurgia",
    "Sessuologia": "Sessuologia",
    "Chirurgia toracica": "Chirurgia toracica",
    "Chirurgia plastica e ricostruttiva": "Chirurgia plastica e ricostruttiva",
    "Malattie infettive": "Malattie infettive",
    "Psichiatria": "Psichiatria",
    "Reumatologia": "Reumatologia",
    "Medicina del lavoro": "Medicina del lavoro",
    "Radiologia interventistica": "Radiologia interventistica",
    "Medicina termale": "Medicina termale",
    "Chirurgia generale": "Chirurgia generale",
    "Chirurgia oncologica": "Chirurgia oncologica",
    "Medicina nucleare": "Medicina nucleare",
    "Neuropsichiatria infantile": "Neuropsichiatria infantile",
    "Medicina trasfusionale": "Medicina trasfusionale",
    "Anestesia e rianimazione": "Anestesia e rianimazione",
    "Dermatologia e venereologia": "Dermatologia e venereologia",
    "Neurologia": "Neurologia",
    "Medicina interna": "Medicina interna",
    "Colonproctologia": "Colonproctologia",
    "Pediatria (Pediatri di libera scelta)": "Pediatria",
    "Gastroenterologia e endoscopia digestiva": "Gastroenterologia e endoscopia digestiva",
    "Terapia del dolore": "Terapia del dolore",
    "Medicina del nuoto e attività subacquee": "Medicina del nuoto e attività subacquee",
    "Chirurgia odontostomatologica": "Chirurgia odontostomatologica",
    "Farmacologia": "Farmacologia",
    "Cardiochirurgia": "Cardiochirurgia",
    "Radioterapia": "Radioterapia",
    "Gnatologia clinica": "Gnatologia clinica",
    "Medicina aeronautica e spaziale": "Medicina aeronautica e spaziale",
    "Patologia clinica": "Patologia clinica",
    "Medicina manuale osteopatica": "Medicina manuale osteopatica",
    "Ginecologia e ostetricia": "Ginecologia e ostetricia",
    "Anatomia patologica": "Anatomia patologica"
}

mapping_udf = udf(lambda x: map_category(x, category_mapping), StringType())
locations_udf = udf(lambda x: get_coordinates(x, "IT"), ArrayType(FloatType()))

In [5]:
medicitalia_doctors_df = load_dataframe(spark, '../medicitalia_doctors/data')
medicitalia_doctors_df = medicitalia_doctors_df.drop_duplicates(['URL'])
medicitalia_doctors_df = medicitalia_doctors_df.na.drop(how='any', subset=['name', 'specialization'])
medicitalia_doctors_df = medicitalia_doctors_df.withColumn('specialization', mapping_udf(medicitalia_doctors_df['specialization']))
medicitalia_doctors_df = medicitalia_doctors_df.where(~col('specialization').contains('MISSING'))
#replace default image with null
default_pictures = medicitalia_doctors_df.groupBy('picture').count().where(col('count')>1).collect()
for dp in default_pictures:
    medicitalia_doctors_df = medicitalia_doctors_df.replace(dp['picture'], None, ['picture'])
#get location coordinates
medicitalia_doctors_df = medicitalia_doctors_df.groupby(lower(col('location')).alias('location')).agg(collect_list(struct(col('URL'), col('name'), col('specialization'), col('ranking'), col('n_replies'), col('n_likes'), col('picture'))).alias('aggregate'))
medicitalia_doctors_df = medicitalia_doctors_df.withColumn('coordinates', locations_udf(medicitalia_doctors_df['location']))
medicitalia_doctors_df = medicitalia_doctors_df.select('location', 'coordinates', explode('aggregate').alias("aggregate"))
medicitalia_doctors_df = medicitalia_doctors_df.select('location', 'coordinates', col('aggregate.URL'), col('aggregate.name'), col('aggregate.specialization'), col('aggregate.ranking'), col('aggregate.n_replies'), col('aggregate.n_likes'), col('aggregate.picture'))


../medicitalia_doctors/data loaded
root
 |-- URL: string (nullable = true)
 |-- name: string (nullable = true)
 |-- specialization: string (nullable = true)
 |-- location: string (nullable = true)
 |-- ranking: double (nullable = true)
 |-- n_replies: double (nullable = true)
 |-- n_likes: double (nullable = true)
 |-- picture: string (nullable = true)



In [ ]:
medicitalia_doctors_df.write.json('../medicitalia_doctors/json_medicitalia_doctors/doctors', mode="overwrite")